In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 8.61 s, sys: 6.03 s, total: 14.6 s
Wall time: 8.64 s


In [2]:
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()
client

Starting new cluster.


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.0c614b7c194e4219993572aa75929e24/status,


In [3]:
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [4]:
dsvh, dsvv = load_data_sen1(dc, date_range, coordinates)

<xarray.Dataset>
Dimensions:      (time: 33, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-06T22:46:14.500000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    vv           (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    vh           (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [5]:
average_vv = calculate_average(dsvv, time_pattern='1M')
progress(average_vv)

VBox()

In [6]:
average_vv.compute()

<xarray.DataArray 'vv' (time: 13, y: 8874, x: 9902)>
array([[[0.06906006, 0.07957305, 0.09409785, ..., 0.23094253,
         0.23027407, 0.23319586],
        [0.06906006, 0.07957305, 0.09409785, ..., 0.23425393,
         0.23664413, 0.2474947 ],
        [0.07378212, 0.08650854, 0.10212713, ..., 0.24990971,
         0.23486657, 0.24579485],
        ...,
        [0.07391845, 0.07370436, 0.07163539, ..., 0.04897952,
         0.04920945, 0.04933887],
        [0.07343939, 0.07252609, 0.0702525 , ..., 0.04814997,
         0.0479173 , 0.04799423],
        [0.07355063, 0.07199537, 0.06989169, ..., 0.04814997,
         0.0479173 , 0.04799423]],

       [[0.00899241, 0.00861329, 0.00829276, ..., 0.23408818,
         0.2404511 , 0.24306983],
        [0.00899241, 0.00861329, 0.00829276, ..., 0.25091022,
         0.25812387, 0.2747655 ],
        [0.00900787, 0.00860327, 0.00830202, ..., 0.2770343 ,
         0.27153593, 0.29995093],
...
        [0.06728882, 0.07021901, 0.07160655, ..., 0.04366349,
         0.04320138, 0.04254311],
        [0.07578255, 0.07568256, 0.07622363, ..., 0.04360874,
         0.0431992 , 0.04247067],
        [0.08462474, 0.08165391, 0.08174166, ..., 0.04360874,
         0.0431992 , 0.04247067]],

       [[0.10820508, 0.11146464, 0.11587781, ..., 0.23969741,
         0.23525833, 0.22797333],
        [0.10820508, 0.11146464, 0.11587781, ..., 0.25590563,
         0.25230905, 0.26266778],
        [0.11237547, 0.11584786, 0.12045314, ..., 0.28987873,
         0.26331538, 0.29312095],
        ...,
        [0.11153916, 0.13004702, 0.12090257, ..., 0.05369304,
         0.05208092, 0.04936117],
        [0.10719641, 0.11629926, 0.11002686, ..., 0.05484349,
         0.05366876, 0.05158882],
        [0.10319728, 0.10290676, 0.09967952, ..., 0.05484349,
         0.05366876, 0.05158882]]], dtype=float32)
Coordinates:
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
  * time         (time) datetime64[ns] 2022-09-30 2022-10-31 ... 2023-09-30
Attributes:
    units:         intensity
    nodata:        nan
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [7]:
coord_time = np.arange(13)
coord_x = average_vv.coords['x']
coord_y = average_vv.coords['y']

In [12]:
full_dataset = np.load('input/saved_datasets/gb_full_dataset_filled.npz')
ndvi = full_dataset['ndvi']
vh = full_dataset['vh']
vv = full_dataset['vv']

In [13]:
ndvi_xarray = xr.DataArray(ndvi, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vh_xarray = xr.DataArray(vh, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vv_xarray = xr.DataArray(vv, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})

In [17]:
train_path = "input/train_points/1130/ST_1130_points.shp"  

# cấu hình nhãn dữ liệu 
label_mapping = {
    "Lua tom": "0",
    "Lua": "1",
    "CHN": "2",
    "CLN": "3",
    "TS": "4",
    "Song": "5",
    "Dat xay dung": "6",
    "Rung": "7",
}

labels = load_data_geo(train_path)
labels.head()

,No,X,Y,LU2022,Hientrang,HT_code,geometry
0,1.0,603860.819,1081162.862,Pomelo,CLN,3,POINT (603860.819 1081162.862)
1,2.0,601306.410,1082782.940,Pomelo,CLN,3,POINT (601306.410 1082782.940)
2,3.0,601084.510,1081351.870,Pomelo,CLN,3,POINT (601084.510 1081351.870)
3,4.0,602193.760,1079205.220,Pomelo,CLN,3,POINT (602193.760 1079205.220)
4,5.0,602459.000,1080946.000,Pomelo,CLN,3,POINT (602459.000 1080946.000)


In [18]:
# Extract data points and create the dataset
data_points = extract_data_point(labels, ndvi_xarray, vh_xarray, vv_xarray)
X_train, y_train = create_dataset(data_points)

In [19]:
 X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [37]:
%%time
best_model, best_params, best_score, cv_results = find_best_classifier(X_train, 
                                                                       y_train, classifiers['naive_bayes'], 
                                                                       param_grids_classifier['naive_bayes'])

# Print best parameters, best result, and MSE over splits
print(f"Best parameters: {best_params}")
print(f"Best result : {best_score}")

Best parameters: {}
Best result : 0.8060617180757443
CPU times: user 166 ms, sys: 3.65 ms, total: 170 ms
Wall time: 209 ms


In [38]:
y_pred = best_model.predict(X_test)

# Calculate Mean Squared Error
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")

Accuracy: 0.7802359882005899


In [36]:
folder_path = 'output/classifiers/'
file_path = os.path.join(folder_path, 'sclassify.pkl')
joblib.dump(best_model, file_path)

['output/classifiers/svm_classify.pkl']

In [ ]:
client.close()
cluster.close()

In [ ]:
plt.imshow(average_ndvi.isel(time=6), vmin=None, vmax=None)
average_ndvi.shape

SAVE THE DATA FOR TRAINING SERVERAL TIME

LOAD SAVED DATASET TO USE

In [ ]:
full_dataset = np.load('input/saved_datasets/full_dataset_cloud.npz')

In [ ]:
ndvi = full_dataset['nvdi']
vh = full_dataset['vh']
vv = full_dataset['vv']

In [ ]:
mask = ~np.isnan(ndvi)
X_train = np.stack([vh[mask], vv[mask]], axis=1)
y_train = ndvi[mask]

In [ ]:
# np.savez('datasets/train_dataset/full_data_regressor_train.npz', data=X_train, label=y_train)

In [ ]:
num_samples = 2000000

# Generate random indices
random_indices = np.random.choice(X_train.shape[0], size=num_samples, replace=False)

# Subset the data using the random indices
X_train_subset = X_train[random_indices]
y_train_subset = y_train[random_indices]

# Check the new shape
print(X_train_subset.shape)
print(y_train_subset.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)

TRAIN MODEL

In [ ]:
%%time
best_model, best_params, best_score, cv_results = find_best_regression(X_train, 
                                                                       y_train, regressors['gradient_boosting'], 
                                                                       param_grids_regressor['gradient_boosting'])

# Print best parameters, best result, and MSE over splits
print(f"Best parameters: {best_params}")
print(f"Best result (negative MSE): {-best_score}")

In [ ]:
folder_path = 'output/regressors/'
file_path = os.path.join(folder_path, 'gb_cloud.pkl')
joblib.dump(best_model, file_path)

In [ ]:
y_pred = best_model.predict(X_test)
# Calculate R² score
r2 = r2_score(y_test, y_pred)
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")

FILL CLOUD 

In [ ]:
best_model = joblib.load("output/regressors/gb_cloud.pkl")

In [ ]:
filled_ndvi = np.copy(ndvi)

In [ ]:
X_pred = np.stack([vh[~mask], vv[~mask]], axis=1)
filled_ndvi[~mask] = best_model.predict(X_pred)

In [ ]:
np.savez('input/saved_datasets/gb_full_dataset_filled.npz', ndvi=filled_ndvi, vv=vv, vh=vh)